In [13]:
import advent
advent.scrape(2024, 22)
data = advent.get_lines(22)
data: list[int] = [int(l) for l in data]

In [14]:
def evolve(num: int):
    num = num ^ (num << 6) & 16777215
    num = num ^ (num >> 5) & 16777215
    num = num ^ (num << 11) & 16777215
    return num
    
def evolve_n_times(num: int, n: int):
    for _ in range(n):
        num = evolve(num)
    return num

result = 0
for line in data:
    result += evolve_n_times(line, 2000)
print(result)

20068964552


In [52]:
# Let's calculate the sequences a 2d matrix
# y-axis length is 2001: the initial position as well as 2000 new positions
import numpy as np
seq = np.zeros((len(data), 2001), dtype=int)
for i, num in enumerate(data):
    for j in range(2001):
        seq[i, j] = num % 10
        num = evolve(num)
# This is size n x 2000: only the 2000 diffs
diffs = np.diff(seq)

difftuples: list[list[tuple[int, int, int, int]]] = [0] * len(data)
difftuplesset: list[set[tuple[int, int, int, int]]] = [{}] * len(data)
for i in range(len(data)):
    difftuples[i] = [tuple(diffs[i, j:(j+4)]) for j in range(1997)]
    difftuplesset[i] = set(difftuples[i])

# Reminder: difftuples[n][i] is the diffs from i,i+1,i+2,i+3,i+4
# To get the value at the end of these diffs, do seq[n][i+4]

In [ ]:
def calculate_coins(tup: tuple[int, int, int, int]) -> int:
    result = 0
    for i in range(len(data)):
        # Use a set here because I expect this to be false the majority of the time
        # and 'x in set' is much faster than 'list.index'
        if tup in difftuplesset[i]:
            ix = difftuples[i].index(tup)
            result += seq[i][ix+4]
    return result

from tqdm import tqdm

mostbananas = 0
# Took like 3m15s lol. A lot of wasted effort, e.g.
# The diffs [-9, -9, -9, -9] is not even possible to ever get
for i in tqdm(range(-9, 10)):
    for j in range(-9, 10):
        for k in range(-9, 10):
            for l in range(-9, 10):
                score = calculate_coins((i, j, k, l))
                if score > mostbananas:
                    mostbananas = score
print(mostbananas)

100%|██████████| 19/19 [03:12<00:00, 10.14s/it]

2246
